## C S 329E Term Project
## Enya Liu

My dataset is about Tokoyo's pressure and climate change from 1875 to 2000.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# df_t = pd.read_csv('t_tokyo.csv')
df_t = pd.read_csv(r'C:\Users\Ley\Desktop\t_tokyo.csv')
df_t

,TOKYO,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1825*,138,135,154,189,207,226,251,244,230,208,175,144
1,1826*,-999,138,187,191,217,249,271,263,233,200,178,147
2,1827*,136,-999,-999,-999,-999,-999,-999,-999,242,204,170,150
3,1828*,139,134,155,182,211,227,251,-999,-999,-999,-999,-999
4,1829,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,1996,66,54,92,127,181,226,262,260,224,180,132,93
172,1997,68,70,105,152,192,227,266,270,229,187,143,92
173,1998,53,70,101,163,205,215,253,272,244,201,139,90
174,1999,66,67,101,150,199,228,259,285,262,195,142,90


In [35]:
# clean up dataset
# change TOKYO to years
df_t = df_t.rename(columns={'TOKYO':'year'})
# eliminate unknowns (-999)
df_t = df_t[(df_t['JAN'] != -999) & (df_t['FEB'] != -999) & (df_t['MAR'] != -999) & (df_t['APR'] != -999) & (df_t['MAY'] != -999)
         & (df_t['JUN'] != -999) & (df_t['JUL'] != -999) & (df_t['AUG'] != -999) & (df_t['SEP'] != -999) & (df_t['OCT'] != -999)
         & (df_t['NOV'] != -999) & (df_t['DEC'] != -999)]
# remove years with uncertain data and incontinuous years
df_t = df_t[(df_t['year'] != '1825*') & (df['year'] != '1839')&(df['year'] != '1840')&(df['year'] != '1841')].reset_index(drop=True)


,year,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1845,63,56,94,140,186,217,256,235,215,169,112,47
1,1846,45,50,79,140,176,222,249,265,257,184,104,76
2,1847,52,51,83,156,185,205,258,263,229,172,129,86
3,1848,48,61,102,150,190,211,271,282,246,178,119,83
4,1850,46,54,102,146,198,220,260,256,257,181,126,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,1996,66,54,92,127,181,226,262,260,224,180,132,93
133,1997,68,70,105,152,192,227,266,270,229,187,143,92
134,1998,53,70,101,163,205,215,253,272,244,201,139,90
135,1999,66,67,101,150,199,228,259,285,262,195,142,90


My research question is that what is the mean temperature in 2022 in Tokyo according to this dataset?

In [ ]:
# find the mean temperature of each year first
df_t['mean_temp'] = df_t.mean(axis = 1)
